# UCS Manager Python SDK Examples

Used with the UCS Platform Emulator with the IP address `192.168.72.4`.

- Update the `ip`, `username`, and `password` constant values below, as necessary.

---

## Constants

In [1]:
# UCSM credentials
IP = '192.168.72.4'
USERNAME = 'admin'
PASSWORD = 'admin'
SECURE = False

# NTP server
NTP_SERVER = '172.16.20.5'

#DNS server
DNS_SERVER = '8.8.8.8'

---

## Connect to UCS

In [2]:
from ucsmsdk.ucshandle import UcsHandle

UCSM_CONN = {
  'ip': IP,
	'username': USERNAME,
	'password': PASSWORD,
  'secure': SECURE
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

True

---

## Review handle object attributes

In [3]:
vars(handle)

print(f'UCS Name: {handle.ucs}')
print(f'IP: {handle.ip}')
print(f'Session Cookie: {handle.cookie}')

UCS Name: ucspe
IP: 192.168.72.4
Session Cookie: 1620823796/28268438-5a2f-4f57-86cf-8d4bc4092e2f


---

## Query and display all compute blade objects

In [4]:
blades = handle.query_classid('computeBlade')

for blade in blades:
    print(blade)


Managed Object			:	ComputeBlade
--------------
class_id                        :ComputeBlade
admin_power                     :policy
admin_state                     :in-service
asset_tag                       :
assigned_to_dn                  :
association                     :none
availability                    :available
available_memory                :49152
chassis_id                      :3
check_point                     :discovered
child_action                    :None
conn_path                       :A,B
conn_status                     :A,B
descr                           :
discovery                       :complete
discovery_status                :
dn                              :sys/chassis-3/blade-1
flt_aggr                        :0
fsm_descr                       :
fsm_flags                       :
fsm_prev                        :DiscoverSuccess
fsm_progr                       :100
fsm_rmt_inv_err_code            :none
fsm_rmt_inv_err_descr           :
fsm_rmt_inv_rslt 

---

## Display specific blade attributes

In [5]:
for blade in blades:
  print(blade.dn, blade.num_of_cpus, blade.available_memory)

sys/chassis-3/blade-1 2 49152
sys/chassis-3/blade-3 2 49152
sys/chassis-3/blade-7 2 49152
sys/chassis-3/blade-5 2 49152
sys/chassis-4/blade-1 2 49152
sys/chassis-4/blade-2 2 49152
sys/chassis-5/blade-1 2 49152
sys/chassis-5/blade-2 2 49152
sys/chassis-5/blade-3 4 49152


---

## SDK Query Methods

### Get objects from a single class

Returns a list of objects: `query_classid`

In [6]:
handle.query_classid('computeBlade')

---

### Get objects from multiple classes

Return a dictionary where `classid` is the key and objects are values: `query_classids`

In [7]:
handle.query_classids('computeBlade', 'computeRackUnit')

{'ComputeBlade': [<ucsmsdk.mometa.compute.ComputeBlade.ComputeBlade at 0x7f9c31756ee0>,
 'ComputeRackUnit': [<ucsmsdk.mometa.compute.ComputeRackUnit.ComputeRackUnit at 0x7f9c31756d00>,
  <ucsmsdk.mometa.compute.ComputeRackUnit.ComputeRackUnit at 0x7f9c3174d940>]}

---

### Get a single object by DN

Return an object: `query_dn`

In [8]:
handle.query_dn('sys/chassis-3/blade-1')

---

### Get multiple objects by DN

Return a dictionary where the DN is the key and the object is the value: `query_dns`

In [9]:
handle.query_dn('sys/chassis-3/blade-1', 'sys/chassis-3/blade-3')

---

### Get child objects for a specific object

Return a list of child objects: `query_children`

In [10]:
handle.query_children(handle.query_dn('sys/chassis-3/blade-1'))

---

## Get Server DN and LED State

In [11]:
# Get all server objects, using classes
servers = handle.query_classids('computeBlade', 'computeRackUnit')

# Loop over all servers
for server_class in servers.values():
    # Loop over each server class object
    for server in server_class:
        # Assign the equipmentLocatorLed class data to a variable
        led = handle.query_children(server, class_id='equipmentLocatorLed')
        print(server.dn, led[0].oper_state)

sys/chassis-3/blade-1 on
sys/chassis-3/blade-3 on
sys/chassis-3/blade-7 on
sys/chassis-3/blade-5 on
sys/chassis-4/blade-1 on
sys/chassis-4/blade-2 on
sys/chassis-5/blade-1 off
sys/chassis-5/blade-2 off
sys/chassis-5/blade-3 off
sys/rack-unit-1 off
sys/rack-unit-2 off
sys/rack-unit-3 off
sys/rack-unit-4 off
sys/rack-unit-5 on
sys/rack-unit-6 off
sys/rack-unit-7 on
sys/rack-unit-8 off
sys/rack-unit-9 off
sys/rack-unit-10 off


---

## Chanege the state of all LEDs

In [12]:
# Get a list of all server objects, by class
# Each class becomes its own dictionary key
server_classes = handle.query_classids('computeBlade', 'computeRackUnit')

# Loop over each server class object list (computeBlade and computeRackUnit)
for server_list in server_classes.values():
      # Loop over each server object within the list
      for server in server_list:
          # Get server LED status
          led = handle.query_children(server, class_id='equipmentLocatorLed')
          # Set previous LED status
          prev_led_status = led[0].oper_state

          # Set the new LED status
          if led[0].oper_state == 'off':
              led[0].admin_state = 'on'
          else:
            led[0].admin_state = 'off'

          # Set the handle MO value and commit the change
          handle.set_mo(led[0])
          handle.commit()
        
          # Display output
          print(f'DN {server.dn}:\n'
                f'\tPrevious - {prev_led_status}\n'
                f'\tNew - {led[0].admin_state}')

DN sys/chassis-3/blade-1:
	Previous - on
	New - off
DN sys/chassis-3/blade-3:
	Previous - on
	New - off
DN sys/chassis-3/blade-7:
	Previous - on
	New - off
DN sys/chassis-3/blade-5:
	Previous - on
	New - off
DN sys/chassis-4/blade-1:
	Previous - on
	New - off
DN sys/chassis-4/blade-2:
	Previous - on
	New - off
DN sys/chassis-5/blade-1:
	Previous - off
	New - on
DN sys/chassis-5/blade-2:
	Previous - off
	New - on
DN sys/chassis-5/blade-3:
	Previous - off
	New - on
DN sys/rack-unit-1:
	Previous - off
	New - on
DN sys/rack-unit-2:
	Previous - off
	New - on
DN sys/rack-unit-3:
	Previous - off
	New - on
DN sys/rack-unit-4:
	Previous - off
	New - on
DN sys/rack-unit-5:
	Previous - on
	New - off
DN sys/rack-unit-6:
	Previous - off
	New - on
DN sys/rack-unit-7:
	Previous - on
	New - off
DN sys/rack-unit-8:
	Previous - off
	New - on
DN sys/rack-unit-9:
	Previous - off
	New - on
DN sys/rack-unit-10:
	Previous - off
	New - on


---

## Query the locator LED status for all servers, reverse the setting, and display results

In [13]:
# Get lists of all servers (returns a dict)
servers = handle.query_classids('computeBlade', 'computeRackUnit')

# Loop over each of the dict values (separate lists of server classes)
for server_classes in servers.values():
# Loop over each individual server class

    for server in server_classes:
        # Get the LED subclass status for each server
        led = handle.query_children(server, class_id='EquipmentLocatorLed')

        # Get the current LED properties
        led_dn = led[0].dn
        led_oper_state = led[0].oper_state

    # Reverse the LED state
    if led_oper_state == 'off':
    	led[0].admin_state = 'on'
    else:
      led[0].admin_state = 'off'
     
		# Set the new LED state
    handle.set_mo(led[0])
    handle.commit()

    # Get the updated status
    led_new_props = handle.query_dn(led_dn)

    # Display the results
    print(f'Locator LED State for {server.dn}:\n'
          f'\tPrevious state - {led_oper_state}\n'
          f'\tNew state - {led_new_props.admin_state}')

Locator LED State for sys/chassis-5/blade-3:
	Previous state - off
	New state - on
Locator LED State for sys/rack-unit-10:
	Previous state - off
	New state - on


---

## Create a new VLAN

In [14]:
# Import the UcsHandle and FabricVlan modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Connect to the UCSM
UCSM_CONN = {
    'ip': IP,
    'username': USERNAME,
    'password': PASSWORD,
    'secure': SECURE
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

# Get the FabricLanCloud class
lan_cloud = handle.query_classid('FabricLanCloud')

# Display the lan_cloud DN
# lan_cloud[0].dn

# Create a MO for the VLAN
vlan_mo = FabricVlan(lan_cloud[0].dn, name='vlan998', id='998')

# Add the VLAN MO to the UCS
handle.add_mo(vlan_mo)
# handle.add_mo(mo=vlan_mo, modify_present=True)
handle.commit()

---

## Create multiple VLANs in a transaction

In [15]:
# Import the UcsHandle and FabricVlan modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Connect to the UCSM
UCSM_CONN = {
    'ip': IP,
    'username': USERNAME,
    'password': PASSWORD,
    'secure': SECURE
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

# Create a FabricLanCloud MO
lan_cloud = handle.query_classid('FabricLanCloud')

# Loop over a range of VLAN numbers
for vlan in range (300, 303):
    # Create a MO for each VLAN
    vlan_mo = FabricVlan(
        parent_mo_or_dn=lan_cloud[0].dn,
        name=f'vlan{vlan}',
        id=str(vlan)
  )

    # Add each MO to the handle object
    handle.add_mo(vlan_mo)

# Commit the changes to UCSM
handle.commit()

---

## Delete multiple VLANs in a transaction

In [16]:
# Import modules
from ucsmsdk.ucshandle import UcsHandle
from ucsmsdk.mometa.fabric.FabricVlan import FabricVlan

# Log in
UCSM_CONN = {
    'ip': IP,
    'username': USERNAME,
    'password': PASSWORD,
    'secure': SECURE
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

# Create a MO for all VLANs
vlans_mo = handle.query_classid('FabricVlan')

# Loop over the list of VLANs (in the vlans_mo), and remove them, except for VLAN 1
for vlan_mo in vlans_mo:
    if vlan_mo.id != '1':
	    handle.remove_mo(vlan_mo)

handle.commit()

---

## UCSM time zone and NTP settings

### Import UcsHandle

In [17]:
from ucsmsdk.ucshandle import UcsHandle

# Login
UCSM_CONN = {
    'ip': IP,
    'username': USERNAME,
    'password': PASSWORD,
    'secure': SECURE
}

handle = UcsHandle(**UCSM_CONN)
handle.login()

True

---

### Get time zones

In [18]:
# Import the CommDateTime class
from ucsmsdk.mometa.comm.CommDateTime import CommDateTime

# Query the CommDateTime DN that is a reference to the 'Timezone-managed' object
timezone_mo = handle.query_dn('sys/svc-ext/datetime-svc')
# timezone_mo = handle.query_classid('CommDateTime') # Also works the same way

print(timezone_mo)


Managed Object			:	CommDateTime
--------------
class_id                        :CommDateTime
admin_state                     :enabled
child_action                    :None
config_state                    :success
date                            :1969-12-31T18:00:00.000
descr                           :
dn                              :sys/svc-ext/datetime-svc
int_id                          :35913
name                            :
oper_port                       :0
oper_timezone                   :America/Chicago
policy_level                    :0
policy_owner                    :local
port                            :0
proto                           :none
rn                              :datetime-svc
sacl                            :None
status                          :None
timezone                        :America/Chicago




---

Get NTP

In [19]:
# Import the CommNtpProvider class
from ucsmsdk.mometa.comm.CommNtpProvider import CommNtpProvider

# Create a MO for the NTP objects
ntp_mos = handle.query_classid('CommNtpProvider')

# Display NTP object details
for n in ntp_mos:
    print(n)

---

### Get DNS

In [20]:
# Import classes and create MOs
from ucsmsdk.mometa.comm.CommDns import CommDns
dns_mo = handle.query_classid('CommDns')
print(dns_mo[0])

from ucsmsdk.mometa.comm.CommDnsProvider import CommDnsProvider
dns_mos = handle.query_classid('CommDnsProvider')
print(dns_mos[0])


Managed Object			:	CommDns
--------------
class_id                        :CommDns
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :sys/svc-ext/dns-svc
domain                          :hullhouse.local
int_id                          :35914
name                            :
oper_port                       :0
policy_level                    :0
policy_owner                    :local
port                            :0
proto                           :none
rn                              :dns-svc
sacl                            :None
status                          :None



Managed Object			:	CommDnsProvider
--------------
class_id                        :CommDnsProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :sys/svc-ext/dns-svc/dns-192.168.72.1
hostname                        :
name      

---

### Set Time Zones

In [21]:
# Create a time zone variable
tz = 'America/Chicago'

# Update the time zone MO with the new value
timezone_mo.timezone = tz

# Update the MO
handle.set_mo(timezone_mo)

### Add NTP servers

In [22]:
# Create NTP server veriable
ntp_server = NTP_SERVER

# Create an NTP parent MO
datetime_mo = handle.query_dn('sys/svc-ext/datetime-svc')

# Create an NTP MO
ntp_mo = CommNtpProvider(
	datetime_mo,
    name=NTP_SERVER
)

# Add and commit the MO
handle.add_mo(
	mo=ntp_mo,
    modify_present=True
)
handle.commit()

### Get NTP servers

In [23]:
# Get a new MO for the new NTP provider
ntp_class = handle.query_classid('CommNtpProvider')
ntp_mos = ntp_class

# Display NTP object details
for n in ntp_mos:
    print(n)


Managed Object			:	CommNtpProvider
--------------
class_id                        :CommNtpProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :sys/svc-ext/datetime-svc/ntp-172.16.20.5
hostname                        :
name                            :172.16.20.5
rn                              :ntp-172.16.20.5
sacl                            :None
status                          :None



Managed Object			:	CommNtpProvider
--------------
class_id                        :CommNtpProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :org-root/comm-pol-system/datetime-svc/ntp-172.16.20.5
hostname                        :
name                            :172.16.20.5
rn                              :ntp-172.16.20.5
sacl                            :None
status                          :None




### Remove NTP servers

In [24]:
# Set the NTP server DN
ntp_dn = ntp_mos[0].dn

# Get the NTP DN managed object
ntp_mo = handle.query_dn(ntp_dn)

# Remove the NTP provider
handle.remove_mo(ntp_mo)
handle.commit()

### Get NTP servers

In [25]:
# Get a new MO for the new NTP provider
ntp_class = handle.query_classid('CommNtpProvider')
ntp_mos = ntp_class

# Display NTP object details
for n in ntp_mos:
    print(n)

---

### Add DNS servers

In [26]:
# Get parent DNS service object
dns_svc_mo = handle.query_dn('sys/svc-ext/dns-svc')

# Create a DNS provider MO
dns_provider_mo = CommDnsProvider(dns_svc_mo, name=DNS_SERVER)

# Add and commit the MO to UCSM
handle.add_mo(dns_provider_mo, True)
handle.commit()

---

### Get DNS servers

In [27]:
dns_mos = handle.query_classid('CommDnsProvider')

for d in dns_mos:
    print(d)


Managed Object			:	CommDnsProvider
--------------
class_id                        :CommDnsProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :sys/svc-ext/dns-svc/dns-192.168.72.1
hostname                        :
name                            :192.168.72.1
rn                              :dns-192.168.72.1
sacl                            :None
status                          :None



Managed Object			:	CommDnsProvider
--------------
class_id                        :CommDnsProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :org-root/comm-pol-system/dns-svc/dns-192.168.72.1
hostname                        :
name                            :192.168.72.1
rn                              :dns-192.168.72.1
sacl                            :None
status                          :None



Mana

---

### Remove DNS servers

In [28]:
# Remove the DNS Provider
dns_providers_mo = handle.query_classid('CommDnsProvider')

### Option #1, loop ###
# Loop over the DNS providers and remove the entry for 8.8.8.8
for dns_server in dns_providers_mo:
    if dns_server.name == DNS_SERVER:
        dns_server_mo = dns_server
        handle.remove_mo(dns_server_mo)
        handle.commit()
    
### Option #2, user a filter string in the query_classid method ###
dns_provider_mos = handle.query_classid(
  class_id='CommDnsProvider',
  filter_str='(dn, "sys/svc-ext/dns-svc/*", type="re")'
)

---

### Get DNS servers

In [29]:
dns_mos = handle.query_classid('CommDnsProvider')

for d in dns_mos:
    print(d)


Managed Object			:	CommDnsProvider
--------------
class_id                        :CommDnsProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :sys/svc-ext/dns-svc/dns-192.168.72.1
hostname                        :
name                            :192.168.72.1
rn                              :dns-192.168.72.1
sacl                            :None
status                          :None



Managed Object			:	CommDnsProvider
--------------
class_id                        :CommDnsProvider
admin_state                     :enabled
child_action                    :None
descr                           :
dn                              :org-root/comm-pol-system/dns-svc/dns-192.168.72.1
hostname                        :
name                            :192.168.72.1
rn                              :dns-192.168.72.1
sacl                            :None
status                          :None




---

### Handle query filter

In [30]:
def query_class_id(
    class_id=None,
    filter_str=None,
    hierarchy=False,
    need_response=False,
    timeout=None,
):
    """ Finds an object using it's class id.

        Args:
            class_id (str): class id of the object to be queried for.
            filter_str(str): query objects with specific property with specific value or pattern specifying value.

                    (property_name, "property_value, type="filter_type")

                    property_name: Name of the Property
                    ## example, dn (without quotes) ##

                    property_value: Value of the property (str or regular expression)
                    ## example, "sys/svc-ext/dns-svc/*" (with quotes) ##

                    filter_type: eq - equal to

                                ne - not equal to

                                ge - greater than or equal to

                                gt - greater than

                                le - less than or equal to

                                lt - less than

                                re - regular expression

                                ## example, "re" (with quotes) ##

                    logical filter type: not, and, or
    """

    handle.query_classid(
        class_id=class_id,
        filter_str=filter_str,
        hierarchy=hierarchy,
        need_response=need_response,
        timeout=timeout
    )

---

### Backup UCS configuration

In [31]:
# Import the backup_ucs class
from ucsmsdk.utils.ucsbackup import backup_ucs

# Perform the backup
backup_ucs(
  handle,
  'config-all',
  './',
  'py-sdk-config-all.xml'
)

Downloading: py-sdk-config-all.xml Bytes: 308213
    308213  [100.00Downloading Finished.


In [32]:
def backup_ucs(
    handle,
    backup_type,
    file_dir,
    file_name,
    timeout_in_sec=600,
    preserve_pooled_values=False
):
    """ Docstring:
        backup_ucs helps create and download Ucs backups.

        Args:
            handle (UcsHandle): Ucs Connection handle
            backup_type (str): type of backup
                            i.e. fullstate/config-logical/config-system/config-all
            file_dir (str): directory to download ucs backup file to
            file_name (str): name for the backup file
                            (supported file extension are '.tar.gz' and '.xml')
            timeout_in_sec (number) : time in seconds for which method waits
                                for the backUp file to generate before it exits.
            preserve_pooled_values (boolean): True/False,
                                                False - by default

        Example:
            file_dir = "/home/user/backup"

            file_name = "config_backup.xml"

            backup_ucs(handle, backup_type="config-logical",
                        file_dir=file_dir, file_name=file_name)
    """
    
    backup_ucs(
        handle=handle,
        backup_type=backup_type,
        file_dir=file_dir,
        file_name=file_name,
        timeout_in_sec=timeout_in_sec,
        preserve_pooled_values=preserve_pooled_values
    )

---

### Import UCS configuration

In [33]:
# Import UCS configuration
from ucsmsdk.utils.ucsbackup import import_ucs_backup

# Import configuration
import_ucs_backup(
	handle,
  './',
  'py-sdk-config-all.xml',
  merge=True
)

2021-09-26 06:34:11,267 - ucs - DEBUG - ./py-sdk-config-all.xml


    311296  [101.00